<a href="https://colab.research.google.com/github/Sarthak-kshirsagar/ansible-playbook-gen-ai/blob/main/RAG_Ansible_Storage_Protect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu
!pip install ibm-watsonx-ai --upgrade
!pip install faiss-cpu
!pip install sentence-transformers
!pip install ansible
import os
import subprocess
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import getpass

In [ ]:
repo_url = 'https://github.com/IBM/ansible-storage-protect'
repo_dir = "ansible-storage-protect"
roles_dir = os.path.join(repo_dir, "roles")

if not os.path.exists(repo_dir):
    print(f"Cloning repository from {repo_url}...")
    subprocess.run(["git", "clone", repo_url], check=True)
else:
    print("Repository already cloned.")


def extract_role_contents(roles_directory):
    """
    Traverse the roles directory and extract each role's README.md content.
    Returns a dictionary mapping role names to their README content.
    """
    knowledge_base = {}
    if os.path.exists(roles_directory):
        for role in os.listdir(roles_directory):
            role_path = os.path.join(roles_directory, role)
            if os.path.isdir(role_path):
                readme_file = os.path.join(role_path, "README.md")
                if os.path.exists(readme_file):
                    try:
                        with open(readme_file, "r", encoding="utf-8") as f:
                            content = f.read()
                        knowledge_base[role] = content
                        print(f"Extracted README.md for role: {role}")
                    except Exception as e:
                        print(f"Error reading {readme_file}: {e}")
                else:
                    print(f"README.md not found for role: {role}")
    else:
        print("Roles directory not found!")
    return knowledge_base

knowledge_base = extract_role_contents(roles_dir)
print(f"Extracted content for {len(knowledge_base)} role(s).")


encoder = SentenceTransformer("all-MiniLM-L6-v2")

def build_faiss_index(knowledge_base):
    texts = []
    role_names = []
    for role, content in knowledge_base.items():
        texts.append(content)
        role_names.append(role)
    embeddings = encoder.encode(texts, convert_to_numpy=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, role_names

faiss_index, role_names = build_faiss_index(knowledge_base)
print("FAISS index built for role contents.")


def find_relevant_roles(user_prompt, faiss_index, role_names):
    query_embedding = encoder.encode([user_prompt], convert_to_numpy=True)
    distances, indices = faiss_index.search(query_embedding, k=3)
    return [role_names[idx] for idx in indices[0]]

def get_role_content(role):
    return knowledge_base.get(role, "")


credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your IBM WatsonX API Key: "),
)


def generate_ansible_playbook(user_prompt, model_id):
    matched_roles = find_relevant_roles(user_prompt, faiss_index, role_names)
    print(f"Relevant role(s) found: {matched_roles}")


    roles_data = {}
    for role in matched_roles:
        roles_data[role] = get_role_content(role)
        print(f"Using content from role: {role}")


    instruction = f"""
You are an experienced Ansible expert who strictly produces valid YAML playbooks. I have extracted a set of Ansible roles from a GitHub repository, where each role is represented by its name and the complete content of its README.md file. Note that even though the reference may include up to three roles, you must only select and use the roles that are necessary to fulfill the user request, incorporating only the variables and parameters from those roles.
Your task is to analyze the following reference roles and generate an optimized Ansible playbook that uses only the appropriate role(s) based on the variables listed in each role's content. The output must be a valid, complete Ansible playbook in YAML format following best practices. Do not include any extra commentary or explanation—only output the YAML playbook.
Reference Roles:
{roles_data}

    User Request: {user_prompt}
    """

    parameters = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 8000,
        GenParams.STOP_SEQUENCES: ["<end·of·code>"]
    }

    model = ModelInference(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        # project_id="68ff7a4b-b299-4b8f-9211-63185899dee6"
        project_id = getpass.getpass("Enter your Project Id: ")
    )

    print("Sending instruction to WatsonX AI model...")
    result = model.generate_text(instruction)
    print("================================")
    print(f"Playbook generated by: {model_id}")
    print("================================")
    return result

user_input = "Create a playbook to install a client on remote VMs using the appropriate roles. Once the server is installed it should create the dsm.sys and dsm.opt files"
generated_playbook = generate_ansible_playbook(user_input, "meta-llama/llama-3-1-70b-instruct")
print("Generated Playbook:")
print(generated_playbook)